In [ ]:
import pyvisa as visa
#import visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from labview_automation import LabVIEW
from MultiPyVu import MultiVuServer as mvs
from MultiPyVu import MultiVuClient as mvc

        
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=20, skip_points=10, time_per_point=0.02):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v




import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise
        
def set_Field(field=0, persistent=True):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise
        
def trigger_ac(ac, offs=0.005, amp=1e-6, duration=0.005, wait_after_arm=2):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(offs))
    ac.write("SOUR:WAVE:AMPL {}".format(amp))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(duration))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(wait_after_arm)
    ac.write("SOUR:WAVE:INIT")
    

    
def init_communication(amp=0.00005):
    print("check ac_1: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_1, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    
    print("check ac_2: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_2, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    """
    print("check ac_mid: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_mid, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)"""
    
    dc.apply_current()# Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = amp            # Sets the source current to 0.1 mA
    dc.enable_source() 
    print("enable dc current now, current:" + str(dc))
    print("check nano volt:")
    for i in range(10):
        print(volt.query("fetch?"))
        time.sleep(0.1)
    time.sleep(10)
    dc.shutdown()   
    print("dc current disabled")

    
def do_measurement_three_pulses(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005, round=6, point_per_round=5,
                                wait_after_trigger=15, pulse=None):
    file_name = "data/{}ac_{}uA{}ms_wait{}s_{}.csv".format(sample_id, int(ac_current * 1e6), 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current))

    for i in range(round):
        print("round: " + str(i+1))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)
                elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=ac_current, duration=duration)
                elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_2':
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * ac_current, duration=duration)
                else:
                    print("no ac " + current)
                    raise
                    
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(3)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""



def LVread():
    lv = LabVIEW()
    lv.start() # Launches the active LabVIEW with the listener VI
    with lv.client() as c:
        control_values = {
        
            }
    
        indicators = c.run_vi_synchronous(r"D:\User Data\Cheng Yang\Sam files\_switch\test_labview_python\\PPMS_Status_Eckberg.vi", control_values)
    #  print(indicators['Temp (K)'],indicators['Angle'], indicators['Field'] )
    #lv.kill() # Stop LabVIEW
    T=float(indicators['Temp (K)'])
    Ang=float(indicators['Angle'])
    Field=float(indicators['Field'])
    return T, Ang, Field



    
def do_measurement_one_pulse_FM(sample_id='', ac_current=[1*1e-3], dc_current=100e-6, duration=0.001,
                                wait_after_trigger=1, pulse=None):
    file_name = "data/{}ac_{}ms_wait{}s_{}.csv".format(sample_id, 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("current,ac_source,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{}\n".format(0, 'init', average_v, std_v, dc_current))

    for i in ac_current:
        print("pulse current=:{} A".format(i))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            if current == 'ac_1':
                    trigger_ac(ac_1, offs=i, duration=duration)
            elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * i, duration=duration)
            elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=i, duration=duration)
            elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * i, duration=duration)
            elif current == 'ac_2':
                    trigger_ac(ac_2, offs=i, duration=duration)
            elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * i, duration=duration)
            else:
                    print("no ac " + current)
                    raise
                    
            time.sleep(wait_after_trigger)
            average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
            with open(file_name, 'a') as f:
                f.write("{},{},{},{},{}\n".format(i, current, average_v, std_v, dc_current))
            print(average_v)
            time.sleep(0.1)
                
rm = visa.ResourceManager()
rm.list_resources()


In [ ]:
def SetField(Field=200, rate=100, Mode='persistent'):
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='persistent':
                client.set_field(Field,
                         rate, 
                         client.field.approach_mode.no_overshoot,
                         client.field.driven_mode.persistent)
            elif  Mode=='driven':
                client.set_field(Field,
                         rate, 
                         client.field.approach_mode.no_overshoot,
                         client.field.driven_mode.driven)
            else:
                print("wrong mode")
                
                
def GetField():
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
    
            Field, sF = client.get_field()
        
    return Field, sF 


def SetTemp(Temp=2, rate=10, Mode='fast_settle'):
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='fast_settle':
                client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.fast_settle)
            elif  Mode=='no_overshoot':
                 client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.no_overshoot)
            else:
                print("wrong mode")
def GetTemp():
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
    
            Temp, sT = client.get_temperature()
        
    return Temp, sT


def SetTempStable(Temp=2, rate=10, Mode='fast_settle'):
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='fast_settle':
                client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.fast_settle)
            elif  Mode=='no_overshoot':
                 client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.no_overshoot)
            else:
                print("wrong mode")
            
            
            Flg = 1
            while  Flg:
                time.sleep(60)
                T = client.get_temperature()
                if abs(T[0] - Temp)<0.1:
                    Flg = 0
            
            time.sleep(1)
            
            Flg = 1
            while  Flg:
                time.sleep(60)
                T = client.get_temperature()
                if abs(T[0] - Temp)<0.1:
                    print('Temp is {}'.format(Temp))
                    Flg = 0
                    
def SetFieldStable(Field=200, rate=1):
     with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            client.set_field(Field,
                         rate, 
                         client.field.approach_mode.no_overshoot,
                         client.field.driven_mode.persistent)
                  
            Flg = 1
            while Flg:
                F = client.get_field()
                if abs(F[0]-Field) < 1:
                    Flg = 0
            print("field is stable = {}Oe".format(Field))
            
            


In [ ]:
sensitivity = {1:26, 0.5:25, 0.2:24, 0.1:23, 0.05:22, 0.02:21, 0.01:20, 0.005:19, 0.002:18, 0.001:17, 0.0005:16, 0.0002:15,
               0.0001:14, 0.00005:13, 0.00002:12, 0.00001:11, 0.000005:10, 0.000002:9, 0.000001:8, 0.0000005:7, 0.0000002:6, 0.0000001:5, 0.00000005:4, 0.00000002:3,
               0.00000001:2, 0.000000005:1, 0.000000002:0}


class Lockin:
    def __init__(self, resource, gpib=1):
        self.lock_in = resource.open_resource('GPIB1::{}::INSTR'.format(gpib))

    def change_harmonic(self, harm=1, sens=1):

       
        self.lock_in.write("HARM {}".format(harm))
        self.lock_in.write("SENS {}".format(sensitivity[sens]))

    def lock_in_read(self):
        x = float(self.lock_in.query("OUTP? 1").strip())
        y = float(self.lock_in.query("OUTP? 2").strip())
        r = float(self.lock_in.query("OUTP? 3").strip())
        theta = float(self.lock_in.query("OUTP? 4").strip())
        return x, y, r, theta

    def lock_in_measure(self, count=10, time_step=0.1,
                        wait_before_measure=1,
                        harm=1, sens=0.00001):
#        self.change_harmonic(harm, sens)
        time.sleep(wait_before_measure)
        xs, ys, rs, thetas = [], [], [], []
        for i in range(count):
            x, y, r, theta = self.lock_in_read()
            xs += [x]
            ys += [y]
            rs += [r]
            thetas += [theta]
            time.sleep(time_step)
            
        xs = sorted(xs)
        ys=  sorted(ys)
        rs=  sorted(rs)
        thetas=sorted(thetas)
        
        if len(xs)>30:
            xs = xs[round(1/6*len(xs)): -round(1/6*len(xs))]
            ys = ys[round(1/6*len(ys)): -round(1/6*len(ys))]
            rs = rs[round(1/6*len(rs)): -round(1/6*len(rs))]
            thetas = thetas[round(1/6*len(thetas)): -round(1/6*len(thetas))]

        x_mean, x_std = np.mean(xs), np.std(xs)
        y_mean, y_std = np.mean(ys), np.std(ys)
        r_mean, r_std = np.mean(rs), np.std(rs)
        theta_mean, theta_std = np.mean(thetas), np.std(thetas)
        
        
        return x_mean, y_mean, r_mean, theta_mean, \
               x_std, y_std,r_std, theta_std

In [ ]:
#switch
ac_6221_1_id = 12
dc_2400_id = 24


dc = Keithley2400("GPIB1::{}".format(dc_2400_id))


ac_1 = rm.open_resource('GPIB1::{}::INSTR'.format(ac_6221_1_id))

lock1 = Lockin(rm, 7)
lock2 = Lockin(rm, 8)
lock3 = Lockin(rm, 10)
lock4 = Lockin(rm, 11)

In [ ]:
 {
   "cell_type": "code",
   "execution_count": null,
   "id": "6ff50b48",
   "metadata": {},
   "outputs": [],
   "source": [
    "dummy = np.linspace(0,1,10)\n",
    "I1=np.arange(0,100e-6,2e-6)\n",
    "I2=np.arange(100e-6,200e-6,5e-7)\n",
    "I3=np.arange(200e-6,-100e-6,-2e-6)\n",
    "I4=np.arange(-100e-6,-200e-6,-5e-7)\n",
    "I5=np.arange(-200e-6,50e-6,50e-6)\n",
    "Currs=np.append(I1,I2)\n",
    "Currs=np.append(Currs,I3)\n",
    "Currs=np.append(Currs,I4)\n",
    "Currs=np.append(Currs,I5)\n",
    "\n",
    "\n",
    "for dumm in dummy:\n",
    "\n",
    "    file_name = \"Gang/{}Oe_{}K_{}_nothermalcycle.csv\".format(0, 2, int(time.time()))\n",
    "    Lin=[]\n",
    "    with open(file_name, 'w') as f:\n",
    "        f.write(\"Current,x1,y1\\n\")\n",
    "    for curr in Currs:\n",
    "            time.sleep(0.3)\n",
    "            ac_1.write(\":SOUR:CURR {}\".format(curr))\n",
    "            print(curr*1e6,'uA')\n",
    "            A=lock2.lock_in_measure(count=1, time_step=0, wait_before_measure=0)\n",
    "            with open(file_name, 'a') as f:\n",
    "                    f.write(\"{},{},{}\\n\".format(curr, A[0], A[1]))\n",
    "            Lin.append(A[0])\n",
    "           "
   ]
  },

In [ ]:
dummy = np.linspace(0,1,1)
I1=np.arange(0,100e-6,2e-6)
I2=np.arange(100e-6,200e-6,5e-7)
I3=np.arange(200e-6,-100e-6,-2e-6)
I4=np.arange(-100e-6,-200e-6,-5e-7)
I5=np.arange(-200e-6,50e-6,50e-6)
Currs=np.append(I1,I2)
Currs=np.append(Currs,I3)
Currs=np.append(Currs,I4)
Currs=np.append(Currs,I5)
for dumm in dummy:
    time.sleep(3)
    file_name = "Gang/{}Oe_{}K_n100mTmagnetization_{}.csv".format(0, 2, int(time.time()))
    Lin=[]
    with open(file_name, 'w') as f:
        f.write("Current,x1,y1\n")
    for curr in Currs:
        time.sleep(0.3)
        ac_1.write(":SOUR:CURR {}".format(curr))
#        print(curr*1e6,'uA')
        A=lock2.lock_in_measure(count=1, time_step=0, wait_before_measure=0)
        with open(file_name, 'a') as f:
            f.write("{},{},{}\n".format(curr, A[0], A[1]))
        Lin.append(A[0])

In [ ]:
Field=[-200,-198]
Field

In [ ]:
SetFieldStable(Field=0,rate=100)

In [ ]:
#Field=np.arange(90000,0,-2000)
#Field=np.arange(85000,0,-5000)
#Field=np.arange(0,92000,2000)
Field=[30000,60000,90000]
#A = np.arange(0,40000,4000)
#B = np.arange(40000,62000,1000)
#C = np.arange(62000,94000,4000)
#A = np.arange(90000,63000,-3000)
#B = np.arange(63000,39000,-1000)
#C = np.arange(39000,0,-3000)
#Field = np.concatenate([A,B,C])

#A = np.arange(750e-6,400e-6,-1e-6)
#B = np.arange(400e-6,-400e-6,-20e-6)
#C = np.arange(-400e-6,-750e-6,-1e-6)
#D = np.arange(-750e-6,-400e-6,1e-6)
#E = np.arange(-400e-6,400e-6,20e-6)
#F = np.arange(400e-6,751e-6,1e-6)
#Currs = np.concatenate([A,B,C,D,E,F])
A = np.arange(750e-6,-750e-6,-2e-6)
B = np.arange(-750e-6,752e-6,2e-6)
Currs = np.concatenate([A,B])

T=GetTemp()
print('Temp is {} K'.format(T[0]))
for field in Field:
    SetFieldStable(Field=field,rate=100)
    time.sleep(3)
    file_name = "Gang/{}Oe_{}K_{}.csv".format(field, T[0], int(time.time()))
    Lin=[]
    with open(file_name, 'w') as f:
        f.write("Field,Current,x1,y1,x2,y2,x3,y3,x4,y4\n")
    for curr in Currs:
            time.sleep(0.2)
            ac_1.write(":SOUR:CURR {}".format(curr))
 #           print(curr*1e6,'uA')
            A=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            B=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            C=lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            D=lock4.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{},{},{},{},{}\n".format(field, curr, A[0], A[1], B[0], B[1], C[0], C[1], D[0], D[1]))
            Lin.append(A[0])
# plt.figure()
# plt.xlabel('Current')
# plt.ylabel('lockin')
# plt.plot(Currs,Lin)        

In [ ]:
A=np.arange(60,5,-5)
B=[2]
Temps=np.concatenate([A,B])
#Temps=B
#Temps=[2]

A = np.arange(-10e-6,10e-6,5e-8)
B = np.arange(10e-6,-15e-6,-5e-6)
#A = np.arange(-16e-6,16e-6,16e-8)
#B = np.arange(16e-6,-20e-6,-4e-6)
#C = np.arange(-5e-6,5e-6,1e-7)
#D = np.arange(5e-6,12e-6,2.5e-7)
#E = np.arange(3e-6,6e-6,1.5e-7)
#F = np.arange(6e-6,-8e-6,-2e-6)
#F = np.arange(6e-6,3e-6,-1.5e-7)
#G = np.arange(3e-6,-3e-6,-5e-8)
#H = np.arange(-3e-6,-6.15e-6,-1.5e-7)
Currs = np.concatenate([A,B])
#SetFieldStable(Field=0,rate=100)
#time.sleep(10)

for Temp in Temps:
    SetTempStable(Temp=Temp, rate=10, Mode='fast_settle')
    T=GetTemp()
    print('Temp is {} K'.format(Temp))
    time.sleep(180)
    file_name = "Gang/{}Oe_{}K_{}.csv".format(0, T[0], int(time.time()))
    Lin=[]
    with open(file_name, 'w') as f:
        f.write("Field,Current,x1,y1,x2,y2,x3,y3,x4,y4\n")
    for curr in Currs:
            time.sleep(0.3)
            ac_1.write(":SOUR:CURR {}".format(curr))
 #           print(curr*1e6,'uA')
            A=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            B=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            C=lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            D=lock4.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{},{},{},{},{}\n".format(0, curr, A[0], A[1], B[0], B[1], C[0], C[1], D[0], D[1]))
            Lin.append(A[0])
            
SetFieldStable(Field=0,rate=100)
SetTempStable(Temp=300, rate=10, Mode='fast_settle')

In [ ]:
Field=np.arange(66000,0,-2000)
#Field=np.arange(85000,0,-5000)
#Field=np.arange(0,92000,2000)
#Field=[0]
#A = np.arange(0,40000,4000)
#B = np.arange(40000,62000,1000)
#C = np.arange(62000,94000,4000)
#A = np.arange(90000,63000,-3000)
#B = np.arange(63000,39000,-1000)
#C = np.arange(39000,0,-3000)
#Field = np.concatenate([A,B,C])

A = np.arange(-16e-6,16e-6,16e-8)
B = np.arange(16e-6,-20e-6,-4e-6)
#C = np.arange(-5e-6,5e-6,1e-7)
#D = np.arange(5e-6,12e-6,2.5e-7)
#E = np.arange(3e-6,6e-6,1.5e-7)
#F = np.arange(6e-6,-8e-6,-2e-6)
#F = np.arange(6e-6,3e-6,-1.5e-7)
#G = np.arange(3e-6,-3e-6,-5e-8)
#H = np.arange(-3e-6,-6.15e-6,-1.5e-7)
Currs = np.concatenate([A,B])

T=GetTemp()
print('Temp is {} K'.format(T[0]))
for field in Field:
    SetFieldStable(Field=field,rate=100)
    time.sleep(3)
    file_name = "Gang/{}Oe_{}K_{}.csv".format(field, T[0], int(time.time()))
    Lin=[]
    with open(file_name, 'w') as f:
        f.write("Field,Current,x1,y1,x2,y2,x3,y3,x4,y4\n")
    for curr in Currs:
            time.sleep(2)
            ac_1.write(":SOUR:CURR {}".format(curr))
 #           print(curr*1e6,'uA')
            A=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            B=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            C=lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            D=lock4.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{},{},{},{},{}\n".format(field, curr, A[0], A[1], B[0], B[1], C[0], C[1], D[0], D[1]))
            Lin.append(A[0])
# plt.figure()
# plt.xlabel('Current')
# plt.ylabel('lockin')
# plt.plot(Currs,Lin)     


A=np.arange(2,65,3)
#B=[65,70,300]
#Temps=np.concatenate([A,B])
Temps=A
#Temps=[2]

A = np.arange(-20e-6,20e-6,20e-8)
B = np.arange(20e-6,-25e-6,-5e-6)
#C = np.arange(-5e-6,5e-6,1e-7)
#D = np.arange(5e-6,12e-6,2.5e-7)
#E = np.arange(3e-6,6e-6,1.5e-7)
#F = np.arange(6e-6,-8e-6,-2e-6)
#F = np.arange(6e-6,3e-6,-1.5e-7)
#G = np.arange(3e-6,-3e-6,-5e-8)
#H = np.arange(-3e-6,-6.15e-6,-1.5e-7)
Currs = np.concatenate([A,B])
SetFieldStable(Field=0,rate=100)
time.sleep(10)

for Temp in Temps:
    SetTempStable(Temp=Temp, rate=10, Mode='fast_settle')
    T=GetTemp()
    print('Temp is {} K'.format(Temp))
    time.sleep(180)
    file_name = "Gang/{}Oe_{}K_{}.csv".format(0, T[0], int(time.time()))
    Lin=[]
    with open(file_name, 'w') as f:
        f.write("Field,Current,x1,y1,x2,y2,x3,y3,x4,y4\n")
    for curr in Currs:
            time.sleep(2)
            ac_1.write(":SOUR:CURR {}".format(curr))
 #           print(curr*1e6,'uA')
            A=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            B=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            C=lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            D=lock4.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{},{},{},{},{}\n".format(0, curr, A[0], A[1], B[0], B[1], C[0], C[1], D[0], D[1]))
            Lin.append(A[0])
            